In [2]:
a = rand(10^7)

10000000-element Vector{Float64}:
 0.589726418613196
 0.11318354548238718
 0.7724096156807099
 0.2878548617925345
 0.06945885529607077
 0.4901503678284562
 0.4307495826441481
 0.27609662949784797
 0.6456794150989269
 0.05715489756419245
 ⋮
 0.16686152929576048
 0.14945958749767485
 0.2039877387563176
 0.12834447518820513
 0.21720698361904722
 0.5066379376490836
 0.40388698345461893
 0.26065373652827584
 0.03413342375779915

In [3]:
sum(a)

5.00027494305945e6

In [4]:
@time sum(a)

@time sum(a)

@time sum(a)

  0.021575 seconds (1 allocation: 16 bytes)
  0.004362 seconds (1 allocation: 16 bytes)
  0.005285 seconds (1 allocation: 16 bytes)


5.00027494305945e6

In [5]:
using Pkg 
Pkg.add("BenchmarkTools")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


   Installed BenchmarkTools ─ v1.3.2


    Updating `~/.julia/environments/v1.9/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
    Updating `~/.julia/environments/v1.9/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.3.2
  [682c06a0] + JSON v0.21.4
  [69de0a69] + Parsers v2.5.10
  [aea7be01] + PrecompileTools v1.1.1
  [21216c6a] + Preferences v1.4.0
  [a63ad114] + Mmap
  [9abbd945] + Profile


Precompiling project...


  ✓ Preferences


  ✓ PrecompileTools


  ✓ Parsers


  ✓ JSON


  ✓ BenchmarkTools
  5 dependencies successfully precompiled in 35 seconds. 10 already precompiled.


In [6]:
using BenchmarkTools

# C

In [10]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i=0; i<n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f 
    print(f, C_code)
end 

c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [12]:
c_sum(a) ≈ sum(a)

true

In [25]:
c_bench = @benchmark $c_sum($a)

BenchmarkTools.Trial: 328 samples with 1 evaluation.
 Range (min … max):  12.131 ms …  18.569 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.424 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.252 ms ± 839.869 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                       ▂   ▃ ▅▂▂▅▆▅▇▃█ ▄▂▂▂▂    
  ▃▃▃▃▁▁▁▃▃▃▁▃▃▃▃▃▃▁▁▄▁▁▃▁▁▃▁▃▄▅▃▃▃▃▆█▆█▆▅██████████████████▇▄ ▄
  12.1 ms         Histogram: frequency by time         16.3 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [35]:
D = Dict()
D["C"] = minimum(c_bench.times) / 1e6

12.130617

# C ffast-math

In [20]:
const Clib_fastmath = tempname()

open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f 
    print(f, C_code)
end 

c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum_fastmath (generic function with 1 method)

In [21]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

BenchmarkTools.Trial: 1011 samples with 1 evaluation.
 Range (min … max):  3.825 ms … 48.580 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.318 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.929 ms ±  2.244 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄▇█▇▅▃▃▂▁                                                  
  ▅███████████▆█▆▆▅▇▆▆▆▆▅▇█▇▅█▄▆▆▆▇█▇▇▆▅▇▅▆▆▄▄▅▆▄▁▁▄▄▄▁▄▄▄▁▄ █
  3.82 ms      Histogram: log(frequency) by time     10.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [36]:
D["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6

3.824526

# Python sum

In [23]:
using Pkg 
Pkg.add("PyCall")
using PyCall

   Resolving package versions...


    Updating `~/.julia/environments/v1.9/Project.toml`
  [438e738f] + PyCall v1.95.1
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [24]:
pysum = pybuiltin("sum")
pysum(a)

5.000274943059679e6

In [27]:
py_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  730.756 ms … 818.771 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     744.352 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   753.057 ms ±  30.190 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █        █  ▁  ▁                                            ▁  
  █▁▁▁▁▁▁▁▁█▁▁█▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  731 ms           Histogram: frequency by time          819 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [37]:
D["Python built-in"] = minimum(py_bench.times) / 1e6

730.756473

# Python NumPy

In [29]:
Pkg.add("Conda")
using Conda 
Conda.add("numpy")

   Resolving package versions...


    Updating `~/.julia/environments/v1.9/Project.toml`
  [8f4d0f93] + Conda v1.8.0
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


┌ Info: Running `conda install -y numpy` in root environment
└ @ Conda /Users/manuelmartinezflores/.julia/packages/Conda/kOnIE/src/Conda.jl:127


done
Solving environment: ...working... 

done



# All requested packages already installed.



In [30]:
numpy_sum = pyimport("numpy")["sum"]

numpy_bench = @benchmark $numpy_sum($a)

BenchmarkTools.Trial: 1028 samples with 1 evaluation.
 Range (min … max):  4.017 ms …   7.921 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.885 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.848 ms ± 350.209 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                          ▆▆ ▁   ▁▃▅█▄▄▁                       
  ▄▆▄▅▄▄▄▃▃▂▂▂▁▂▂▁▁▂▁▁▁▂▃▇█████▇████████▆▆▄▄▃▃▃▄▃▃▃▃▃▄▄▃▅▃▃▃▃ ▄
  4.02 ms         Histogram: frequency by time         5.6 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [38]:
D["NumPy"] = minimum(numpy_bench.times) / 1e6

4.017132

# Python Hand

In [32]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a 
    return s 
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x1221e1480>

In [33]:
pyh_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  727.445 ms … 760.671 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     738.667 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   740.742 ms ±  12.269 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █    █   █          █      █                  █             █  
  █▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  727 ms           Histogram: frequency by time          761 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [39]:
D["Python Hand"] = minimum(pyh_bench.times) / 1e6

727.444744

# Julia built-in

In [40]:
@which sum(a)

sum(a::AbstractArray; dims, kw...)
     @ Base reducedim.jl:994

In [41]:
j_bench = @benchmark $sum($a)

BenchmarkTools.Trial: 1123 samples with 1 evaluation.
 Range (min … max):  3.999 ms …   6.114 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.412 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.438 ms ± 118.838 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                          ▄▃▄██▇▂▃▂▃▁       ▁                  
  ▂▁▁▂▁▁▁▁▁▁▁▁▁▂▁▂▂▁▁▂▂▁▃▅████████████▇▆▇▅▇▇█▇▇▇▆▆▄▅▄▃▃▂▂▁▂▂▂ ▄
  4 ms            Histogram: frequency by time        4.75 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [42]:
D["Julia built-in"] = minimum(j_bench.times) / 1e6
D

Dict{Any, Any} with 6 entries:
  "C"               => 12.1306
  "NumPy"           => 4.01713
  "Python built-in" => 730.756
  "Python Hand"     => 727.445
  "C -ffast-math"   => 3.82453
  "Julia built-in"  => 3.99911

# Julia Hand

In [44]:
function mysum(A)
    s = 0.0
    for a in A
        s += a 
    end 
    s
end 

mysum (generic function with 1 method)

In [46]:
jh_bench = @benchmark $mysum($a)

BenchmarkTools.Trial: 343 samples with 1 evaluation.
 Range (min … max):  12.149 ms … 23.315 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.068 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.595 ms ±  1.628 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁     ▃▄▁▁                          ▆█▇                  
  ▃▆▄▄█▇█▇▆▆████▇▆▇▆█▆▇▃▃▆▄▄▄▁▃▁▁▄▁▁▁▃▃▄▅▇████▇▆▅▇▅▁▃▃▁▁▁▁▁▃▃ ▄
  12.1 ms         Histogram: frequency by time        17.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [47]:
D["Julia Hand"] = minimum(jh_bench.times) / 1e6
D

Dict{Any, Any} with 7 entries:
  "Julia Hand"      => 12.1493
  "C"               => 12.1306
  "NumPy"           => 4.01713
  "Python built-in" => 730.756
  "Python Hand"     => 727.445
  "C -ffast-math"   => 3.82453
  "Julia built-in"  => 3.99911

# Julia w. simd

In [48]:
function sum_simd(A)
    s = 0.0
    @simd for a in A
        s += a 
    end 
    s 
end

sum_simd (generic function with 1 method)

In [49]:
jsimd_bench = @benchmark $sum_simd($a)

BenchmarkTools.Trial: 1066 samples with 1 evaluation.
 Range (min … max):  3.741 ms …  18.071 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.482 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.673 ms ± 848.106 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▁▅▇██▇▆▅▅▃▃▂▂▁▂ ▁                                  ▁
  ▆▅▅▆▇▆▆▇▇███████████████▇██▆▇▇█▆█▆▆█▆▄▇▅▄▆▅▄▆▄▄▄▅▁▆▄▄▁▄▁▅▅▅ █
  3.74 ms      Histogram: log(frequency) by time      6.87 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [50]:
D["Julia simd"] = minimum(jsimd_bench.times) / 1e6
D

Dict{Any, Any} with 8 entries:
  "Julia Hand"      => 12.1493
  "C"               => 12.1306
  "NumPy"           => 4.01713
  "Python built-in" => 730.756
  "Python Hand"     => 727.445
  "C -ffast-math"   => 3.82453
  "Julia built-in"  => 3.99911
  "Julia simd"      => 3.74106

In [55]:
for (key, value) in sort(collect(D), by=last)
    println(rpad(key, 20, "."), lpad(round(value; digits=1), 6, "."))
end

Julia simd.............3.7
C -ffast-math..........3.8
Julia built-in.........4.0
NumPy..................4.0
C.....................12.1
Julia Hand............12.1
Python Hand..........727.4
Python built-in......730.8
